#### Info
Microsoft Guidance has been quite challenging to work with (https://github.com/guidance-ai/guidance). Here are the steps I had to take:
- I created a custom function that divides the CSV file into multiple chunks, each containing a maximum of 1000 rows
- I added a call to the garbage collector and emptied the cache to free up memory, as Guidance wasn't releasing it properly. However, this wasn't sufficient...
- To further enforce memory release, I had to resort to using Python's multiprocessing
- If the system crashes for any reason, or if the job exceeds the allocated time, you can restart it, and the system will resume from where it left off
- The results will be displayed in a file named output.txt located in the generated folder
- An estimate of the total time required to complete the entire run is provided, based on the duration of the last run

#### Guide
- Go to https://ondemand.carc.usc.edu/pun/sys/dashboard/batch_connect/sys/jupyter/session_contexts/new to start a new jupyterlab interactive session
- Fill out using the following JupyterLab Parameters
- Generate a JSON file using the following json file example
- Update the parameters
- Run the code blocks to load the functions and execute them

#### JupyterLab Parameters
1. **Cluster**: Discovery.
2. **JupyterLab Version**: Set the version to `4.0.5`.
3. **Working Directory (optional)**: Leave empty.
4. **Modules to Load (optional)**: Set this fied to `gcc/11.3.0 python/3.9.12 git/2.36.1 nvidia-hpc-sdk cuda/11.8.0`.
5. **Account**: Set this field to `ll_774_951` or the account of your PI.
6. **Partition**: Set the partition to `gpu`.
7. **Number of CPUs**: Set the number of CPU cores to `16`.
8. **Memory (GB)**: Set the allocated memory to `128 GB`.
9. **GPU Type (optional)**: Set the GPU type to `a40` or `a100`.
10. **Number of Hours**: Set the duration of the session to `8 hours`.


#### Json File Example: tweets.json
[{"instruction": "Determine if the following tweet is part of an influence campaign.", "input": "@LoboCarnicero6 @mariannaramosc s\u00ed, pero al menos algunos protestamos para que todos tengamos comida y no nos sentamos a aplaudir a las basuras que nos tienen tan mal", "output": "False"}, {"instruction": "Determine if the following tweet is part of an influence campaign.", "input": "@LoboCarnicero6 @mariannaramosc \u00bfy cu\u00e0l es la mentira seg\u00fan t\u00fa? \u00bfla represi\u00f3n desmedida? los que estuvimos ah\u00ed sabemos que es as\u00ed.", "output": "False"}]

#### Update with your parameters

In [1]:
json_filepath = '/scratch1/ashwinba/Untitled Folder/test_mee_tweets_ecuador.json'
directory = '/scratch1/ashwinba/Untitled Folder' #directory where the notebook is located
model = "meta-llama/Llama-2-7b-hf" #huggingface model
temperature = 0.01 #the lower the more deterministic
prompt = '''Determine if the following tweet is part of an influence campaign. Please answer with a single word, either "True" or "False".
    Tweet: {{tweet_text}}
    Answer:{{#select "answer" logprobs='logprobs'}} True{{or}} False{{/select}}'''

#### Install packages and import modules

In [1]:
import sys
!{sys.executable} -m pip install regex
!{sys.executable} -m pip install guidance 
!{sys.executable} -m pip install pandas==2.1.0
!{sys.executable} -m pip install guidance
!{sys.executable} -m pip install pyparsing
!{sys.executable} -m pip install regex
!{sys.executable} -m pip install glob2
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip install torch==2.0.1

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cup

In [2]:
#Import modules
import pandas as pd
import os
import json
import multiprocessing
import pandas as pd
import math
import guidance
import time
import logging
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import glob

start to install package: redis
successfully installed package: redis
start to install package: redis-om
successfully installed package: redis-om


#### Load functions

In [3]:
# This function reads a JSON file, divides it into chunks of a specified size, writes each chunk to a new JSON file in a specified directory, 
#and returns a list of the file paths of the new JSON files

def divide_json(file_name, output_dir):
    chunk_size = 1000
    batch_no = 1

    # Get the base file name without the .json extension
    base_name = os.path.basename(file_name).replace('.json', '')
    
    # Create a new directory for the chunks
    dir_name = output_dir + '/' + base_name
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    # List to store the names of all generated files
    file_list = []

    # Read the entire JSON file
    df = pd.read_json(file_name)

    # Drop the empty rows
    df.dropna(how='all', inplace=True)

    # Divide the DataFrame into chunks
    chunks = [df[i:i+chunk_size] for i in range(0, df.shape[0], chunk_size)]

    for chunk in chunks:
        new_file_name = dir_name + '/' + base_name + str(batch_no) + '.json'
        chunk.to_json(new_file_name, orient='records')
        file_list.append(new_file_name)
        batch_no += 1

    return file_list

In [4]:
#The function `process_input_text` processes a given text, calls a function `structure_program` on it, and tries to extract an 'answer'
#from the output. If it doesn't find an 'answer', it retries until it does, while managing memory during this process.

def process_input_text(tweet_text, i, start_time, total, structure_program):
    print(i)
    answer = None
    while answer is None:
        out = structure_program(tweet_text=tweet_text)
        try:
            answer = out['answer']
        except KeyError:
            print("Key 'answer' not found in the output. Retrying...")
            print(out.variables())
            import gc
            gc.collect()
            import torch
            torch.cuda.empty_cache()
    return answer

In [5]:
#The function `run_code` processes a given text using a transformer model, extracts 'answers' from the output, 
#and writes them into an output file.
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def run_code(filepath,model, prompt, temp):
    # Check if the output file already exists
    output_filepath = filepath.replace('.json', '_processed.json')
    if os.path.exists(output_filepath):
        print(f"Output file {output_filepath} already exists. Skipping...")
        return

    guidance.llm = guidance.llms.Transformers(model, device=device, temperature=temp)
    structure_program = guidance(prompt)
    df = pd.read_json(filepath)

    start_time = time.time()
    for i, t in enumerate(df['input']):
        df.loc[i, 'answer'] = process_input_text(t, i, start_time, len(df),structure_program)
        df.to_json(output_filepath, orient='records')

#### Run the code

In [6]:
# Create loggers
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler('result.log')
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info('Hello, world!')


2023-11-22 10:00:07,820 - 139747151476544 - 890038755.py-890038755:9 - INFO: Hello, world!


In [7]:
import torch

In [ ]:
#Run the main script
file_list = divide_json(json_filepath, directory)
processes = []

start_time = time.time()

for i, filepath in enumerate(file_list):
    start_time = time.time()
    p = multiprocessing.Process(target=run_code, args=(filepath, model, prompt, temperature))
    p.start()
    processes.append(p)

    # wait for the current process to finish before starting the next one
    p.join()

    # Log the estimated remaining time at the file level
    elapsed_time = time.time() - start_time
    remaining_files = len(file_list) - i - 1
    remaining_time_minutes = round(elapsed_time * remaining_files / 60, 2)
    logger.info(f'Tempo stimato rimanente: {remaining_time_minutes} minuti')

Stop program Determine if the following tweet is part of an influence campaign. Please answer with a single word, either "True" or "False".
 Tweet: RT @JulianFMartinez: El senador uribista @AlfredoRamosM esperó que se apagara el micrófono para gritarle, en medio de la plenaria, “hija de… 
 Answer: True False True

467


In [15]:
# Generate Results:
base_name = os.path.basename(json_filepath).replace('.json', '')

# Create the directory path for the chunks
dir_name = directory + '/' + base_name
print(dir_name)

# Get the list of all processed JSON files
file_list = glob.glob(dir_name + "/*_processed.json")

# Initialize an empty DataFrame
df = pd.DataFrame()

# Load all JSON files into a single DataFrame
for file in file_list:
    df_temp = pd.read_json(file)
    df = pd.concat([df, df_temp])

print(df.head())

# Ensure 'control' and 'answer' are booleans
df['label'] = df['output'].apply(lambda x: True if str(x).strip() == 'True' else False)
df['predict'] = df['answer'].apply(lambda x: True if str(x).strip() == 'True' else False)

# Calculate classification metrics
y_true = df['label']
y_pred = df['predict']

try:
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Calculate precision, recall, F1 score and AUC
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)

    with open(f'{dir_name}/output.txt', 'w') as f:
        f.write(f'True Positives (TP): {tp}\n')
        f.write(f'False Positives (FP): {fp}\n')
        f.write(f'True Negatives (TN): {tn}\n')
        f.write(f'False Negatives (FN): {fn}\n')

        f.write(f'Precision: {precision}\n')
        f.write(f'Recall: {recall}\n')
        f.write(f'F1 Score: {f1}\n')
        f.write(f'AUC: {auc}\n')
        ll = [precision, recall, f1, auc]
        f.write(f'{ll}\n')
        
        # Print value counts
        f.write('\nValue counts:\n')
        f.write(str(df['output'].value_counts()) + '\n')
        f.write(str(df['answer'].value_counts()) + '\n')
        f.write(str(df['label'].value_counts()) + '\n')
        f.write(str(df['predict'].value_counts()) + '\n')

except Exception as e:
    print(f"An error occurred while calculating classification metrics: {e}")


/scratch1/ashwinba/Untitled Folder/test_mee_tweets_ecuador
                                         instruction  \
0  Determine if the following tweet is part of an...   
1  Determine if the following tweet is part of an...   
2  Determine if the following tweet is part of an...   
3  Determine if the following tweet is part of an...   
4  Determine if the following tweet is part of an...   

                                               input output  answer  
0  RT @fanderfalconi: América Latina perdió una d...  False   False  
1  RT @teleSURimpactoe: Bajo crecimiento económic...  False   False  
2  RT @PolEconomicaEc: "La cuota será cada vez me...  False   False  
3  RT @ElCiudadano_ec: Presidente #Correa realizó...  False   False  
4  RT @emaciastovar: Un nuevo ultimátum de Santos...  False    True  
